
## Importing necessary libraries



In [3]:

!pip install transformers datasets scikit-learn --quiet

import kagglehub
import pandas as pd
import os
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import DataCollatorWithPadding
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

## 1. Installing and loading the data 

In [4]:
# Download latest version
path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")

print("Path to dataset files:", path)
# List files in the downloaded dataset directory
dataset_files = os.listdir(path)
print("Files:", dataset_files)

Path to dataset files: /kaggle/input/sentiment-analysis-dataset
Files: ['training.1600000.processed.noemoticon.csv', 'train.csv', 'testdata.manual.2009.06.14.csv', 'test.csv']


In [5]:
train_df = pd.read_csv(f"{path}/train.csv", encoding='ISO-8859-1')
test_df = pd.read_csv(f"{path}/test.csv", encoding='ISO-8859-1')

In [6]:
# Check column names
print(train_df.columns)

Index(['textID', 'text', 'selected_text', 'sentiment', 'Time of Tweet',
       'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)',
       'Density (P/Km²)'],
      dtype='object')


## 2. Data Preprocessing

In [7]:
train_df = train_df[['text', 'sentiment']].dropna()
test_df = test_df[['text', 'sentiment']].dropna()

# Convert text to string (in case it's float or something else)
train_df['text'] = train_df['text'].astype(str)
test_df['text'] = test_df['text'].astype(str)

In [8]:
train_df.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [9]:
# Encode labels (make sure they are in the form 0, 1, 2)
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['sentiment'])
test_df['label'] = label_encoder.transform(test_df['sentiment'])

In [10]:
print("Label classes:", label_encoder.classes_)

Label classes: ['negative' 'neutral' 'positive']


In [11]:
train_df.head()

,text,sentiment,label
0,"I`d have responded, if I were going",neutral,1
1,Sooo SAD I will miss you here in San Diego!!!,negative,0
2,my boss is bullying me...,negative,0
3,what interview! leave me alone,negative,0
4,"Sons of ****, why couldn`t they put them on t...",negative,0


## 3. Convert to Hugging Face datasets

In [ ]:
train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label']])
dataset = DatasetDict({'train': train_dataset, 'test': test_dataset})

In [14]:
# Load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
def tokenize(batch):
  texts = [str(t) for t in batch["text"]]
  return tokenizer(texts, padding=True, truncation=True)

In [16]:
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/27480 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

## 4. Loading the model and model training

In [17]:
# Load pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# 6. Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    logging_dir="./logs"
)


# 7. Metrics function
from sklearn.metrics import classification_report

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=1)
    return classification_report(labels, preds, output_dict=True)


# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-18-389e3d044040>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: naren-c-research (naren-c-research-chennai-mathematical-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,0,1,2,Accuracy,Macro avg,Weighted avg
1,0.607500,0.501969,"{'precision': 0.8279329608938547, 'recall': 0.7402597402597403, 'f1-score': 0.7816455696202531, 'support': 1001.0}","{'precision': 0.7431192660550459, 'recall': 0.793006993006993, 'f1-score': 0.7672530446549392, 'support': 1430.0}","{'precision': 0.8382749326145552, 'recall': 0.8458748866727108, 'f1-score': 0.842057761732852, 'support': 1103.0}",0.794567,"{'precision': 0.8031090531878186, 'recall': 0.7930472066464814, 'f1-score': 0.7969854586693481, 'support': 3534.0}","{'precision': 0.7968417359896205, 'recall': 0.7945670628183361, 'f1-score': 0.7946770742042366, 'support': 3534.0}"
2,0.399000,0.522870,"{'precision': 0.7430313588850174, 'recall': 0.8521478521478522, 'f1-score': 0.7938576081898557, 'support': 1001.0}","{'precision': 0.7648766328011611, 'recall': 0.737062937062937, 'f1-score': 0.7507122507122507, 'support': 1430.0}","{'precision': 0.8720238095238095, 'recall': 0.7969174977334542, 'f1-score': 0.8327806726669825, 'support': 1103.0}",0.788342,"{'precision': 0.7933106004033293, 'recall': 0.7953760956480811, 'f1-score': 0.7924501771896963, 'support': 3534.0}","{'precision': 0.7921307971291242, 'recall': 0.7883418222976797, 'f1-score': 0.7885475569519654, 'support': 3534.0}"
3,0.250200,0.633538,"{'precision': 0.7652582159624414, 'recall': 0.8141858141858141, 'f1-score': 0.7889641819941917, 'support': 1001.0}","{'precision': 0.7597684515195369, 'recall': 0.7342657342657343, 'f1-score': 0.7467994310099573, 'support': 1430.0}","{'precision': 0.8371665133394665, 'recall': 0.8250226654578422, 'f1-score': 0.8310502283105022, 'support': 1103.0}",0.785229,"{'precision': 0.7873977269404815, 'recall': 0.7911580713031302, 'f1-score': 0.7889379471048837, 'support': 3534.0}","{'precision': 0.7854801992260252, 'recall': 0.7852292020373515, 'f1-score': 0.7850381251689046, 'support': 3534.0}"
4,0.158000,0.877392,"{'precision': 0.7828232971372162, 'recall': 0.7922077922077922, 'f1-score': 0.7874875868917577, 'support': 1001.0}","{'precision': 0.7422402159244265, 'recall': 0.7692307692307693, 'f1-score': 0.7554945054945055, 'support': 1430.0}","{'precision': 0.8585178055822906, 'recall': 0.8087035358114234, 'f1-score': 0.8328664799253035, 'support': 1103.0}",0.788059,"{'precision': 0.7945271062146445, 'recall': 0.790047365749995, 'f1-score': 0.7919495241038556, 'support': 3534.0}","{'precision': 0.7900268162884975, 'recall': 0.7880588568194681, 'f1-score': 0.7887051343218455, 'support': 3534.0}"
5,0.090000,1.025044,"{'precision': 0.7740058195926285, 'recall': 0.7972027972027972, 'f1-score': 0.7854330708661418, 'support': 1001.0}","{'precision': 0.7415342087076711, 'recall': 0.7503496503496504, 'f1-score': 0.745915884602016, 'support': 1430.0}","{'precision': 0.8475378787878788, 'recall': 0.8114233907524931, 'f1-score': 0.8290875405280222, 'support': 1103.0}",0.782683,"{'precision': 0.7876926356960595, 'recall': 0.7863252794349802, 'f1-score': 0.78681216533206, 'support': 3534.0}","{'precision': 0.7838166452086082, 'recall': 0.7826825127334465, 'f1-score': 0.7830678483645442, 'support': 3534.0}"


Trainer is attempting to log a value of "{'precision': 0.8279329608938547, 'recall': 0.7402597402597403, 'f1-score': 0.7816455696202531, 'support': 1001.0}" of type <class 'dict'> for key "eval/0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7431192660550459, 'recall': 0.793006993006993, 'f1-score': 0.7672530446549392, 'support': 1430.0}" of type <class 'dict'> for key "eval/1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8382749326145552, 'recall': 0.8458748866727108, 'f1-score': 0.842057761732852, 'support': 1103.0}" of type <class 'dict'> for key "eval/2" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8031090531878186, 'recall':

TrainOutput(global_step=4295, training_loss=0.29097850875055137, metrics={'train_runtime': 1272.5488, 'train_samples_per_second': 107.972, 'train_steps_per_second': 3.375, 'total_flos': 3692549052063648.0, 'train_loss': 0.29097850875055137, 'epoch': 5.0})

In [20]:

eval_results = trainer.evaluate()

Trainer is attempting to log a value of "{'precision': 0.8279329608938547, 'recall': 0.7402597402597403, 'f1-score': 0.7816455696202531, 'support': 1001.0}" of type <class 'dict'> for key "eval/0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7431192660550459, 'recall': 0.793006993006993, 'f1-score': 0.7672530446549392, 'support': 1430.0}" of type <class 'dict'> for key "eval/1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8382749326145552, 'recall': 0.8458748866727108, 'f1-score': 0.842057761732852, 'support': 1103.0}" of type <class 'dict'> for key "eval/2" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8031090531878186, 'recall':

In [22]:
print("\nClassification Report:")
predictions = trainer.predict(tokenized_dataset["test"])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(axis=1)
print(classification_report(y_true, y_pred))



Classification Report:


              precision    recall  f1-score   support

           0       0.83      0.74      0.78      1001
           1       0.74      0.79      0.77      1430
           2       0.84      0.85      0.84      1103

    accuracy                           0.79      3534
   macro avg       0.80      0.79      0.80      3534
weighted avg       0.80      0.79      0.79      3534

